In [1]:
import pickle
import gensim
from nltk.corpus import stopwords 

In [2]:
f = open("../Dataset/BOW.txt", "r")

for line in f:
    list_of_words = line.split(',')
f.close()

index_to_words = {}
for i in range(len(list_of_words)):
    index_to_words[i+1] = list_of_words[i]

In [3]:
f = open("../Dataset/dataset.pkl", "rb")
msdid_to_lyrics =  pickle.load(f)
f.close()

msd_ids = list(msdid_to_lyrics.keys())

In [4]:
stop_words = []
languages_list = ['english', 'spanish', 'german', 'french', 'italian']
for language in languages_list:
    stop_words.extend(stopwords.words(language))

from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

stop_words = [ps.stem(w) for w in stop_words]
# print(stop_words)


In [9]:
processed_docs = []

for msd_id in msd_ids:
    lyrics = msdid_to_lyrics[msd_id][-1]

    word_lyrics = []
    for c in lyrics:
        collon_index = c.find(':')
        ind = int(c[:collon_index])
        freq = int(c[collon_index+1:])
        for i in range(freq):
            w = index_to_words[ind]
            if len(w)<=3 or w in stop_words:
                continue
            word_lyrics.append(w)
    processed_docs.append(word_lyrics)

In [13]:
import os
f = open("Processed_Docs.pkl", "wb")
pickle.dump(processed_docs, f)

In [5]:
import os
f = open("Processed_Docs.pkl", "rb")
processed_docs = pickle.load(f)

In [6]:
print(len(processed_docs))

237662


In [7]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [8]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 arrang
1 captur
2 damn
3 devast
4 element
5 fashion
6 flesh
7 grace
8 grant
9 highest
10 ignor


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [11]:
lda_model = gensim.models.LdaModel(bow_corpus, num_topics=3, id2word=dictionary, passes=2, alpha = 'auto')

In [12]:
lda_model.save("lda_Model_topics_3")

In [13]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.8337118773283972


In [14]:
print(lda_model.print_topics())


[(0, '0.059*"love" + 0.032*"know" + 0.028*"babi" + 0.023*"yeah" + 0.020*"want" + 0.015*"girl" + 0.015*"feel" + 0.014*"gonna" + 0.014*"wanna" + 0.014*"make"'), (1, '0.017*"this" + 0.014*"like" + 0.011*"time" + 0.011*"know" + 0.008*"never" + 0.007*"back" + 0.007*"take" + 0.007*"life" + 0.007*"would" + 0.006*"away"'), (2, '0.022*"amor" + 0.013*"vida" + 0.013*"solo" + 0.012*"quiero" + 0.009*"nÃ£o" + 0.009*"mÃ¡s" + 0.008*"piÃ¹" + 0.007*"quando" + 0.006*"nunca" + 0.006*"siempr"')]


In [15]:
res = lda_model.get_document_topics(bow_corpus)

In [16]:
topic_count = []
for i in range(20):
    topic_count.append(0)
list1 = []
for i in range(len(res)):
    a = []
    for c in res[i]:
        if c[1] >0.24999:
            a.append(c[0])
    list1.append(a)
    for j in range(20):
        if j in a:
            topic_count[j] += 1
            
    if(i%10000 == 0):
        print(i)
print(topic_count)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
[78923, 189455, 24391, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
lda_model_2 = gensim.models.LdaMulticore(bow_corpus, num_topics=40, id2word=dictionary, passes=2, workers = 2)

In [22]:
lda_model_2.save("lda_Model_topics_3")

In [23]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_2, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46624006363405435


In [24]:
print(lda_model_2.print_topics())

[(38, '0.094*"call" + 0.061*"better" + 0.053*"chorus" + 0.046*"name" + 0.029*"readi" + 0.029*"hear" + 0.021*"hous" + 0.021*"brother" + 0.018*"line" + 0.016*"number"'), (13, '0.018*"soul" + 0.018*"blood" + 0.017*"burn" + 0.016*"kill" + 0.016*"death" + 0.011*"fire" + 0.010*"this" + 0.009*"dead" + 0.008*"rise" + 0.008*"hell"'), (0, '0.063*"cool" + 0.039*"shut" + 0.032*"niet" + 0.027*"anim" + 0.026*"vain" + 0.025*"fever" + 0.025*"rockin" + 0.025*"maar" + 0.024*"minut" + 0.023*"monkey"'), (7, '0.180*"night" + 0.126*"danc" + 0.111*"fall" + 0.028*"christma" + 0.028*"ladi" + 0.022*"send" + 0.020*"white" + 0.012*"floor" + 0.011*"dream" + 0.009*"miracl"'), (39, '0.088*"look" + 0.078*"around" + 0.060*"everi" + 0.045*"break" + 0.026*"breath" + 0.025*"make" + 0.022*"turn" + 0.017*"enough" + 0.016*"ground" + 0.014*"noth"'), (30, '0.588*"yeah" + 0.151*"wait" + 0.033*"beat" + 0.013*"rhythm" + 0.008*"loos" + 0.007*"long" + 0.006*"time" + 0.006*"kick" + 0.006*"ohoh" + 0.005*"dancin"'), (15, '0.207*"beli

In [25]:
lda_model_3 = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers = 2)

In [26]:
lda_model_3.save("lda_Model_topics_20_alpha_not_def")

In [27]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_3, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5868904872668171


In [28]:
print(lda_model_3.print_topics())

[(0, '0.075*"danc" + 0.040*"fire" + 0.028*"citi" + 0.025*"black" + 0.024*"burn" + 0.024*"move" + 0.022*"parti" + 0.019*"white" + 0.017*"water" + 0.015*"town"'), (1, '0.034*"like" + 0.018*"nigga" + 0.017*"fuck" + 0.017*"this" + 0.015*"shit" + 0.011*"caus" + 0.009*"money" + 0.009*"back" + 0.009*"bitch" + 0.009*"know"'), (2, '0.153*"gonna" + 0.119*"wanna" + 0.063*"give" + 0.063*"make" + 0.053*"gotta" + 0.025*"ride" + 0.024*"turn" + 0.023*"round" + 0.018*"take" + 0.017*"around"'), (3, '0.026*"would" + 0.016*"could" + 0.015*"home" + 0.013*"said" + 0.012*"like" + 0.012*"time" + 0.011*"this" + 0.011*"never" + 0.009*"night" + 0.009*"look"'), (4, '0.030*"amor" + 0.021*"quiero" + 0.019*"vida" + 0.015*"mÃ¡s" + 0.011*"solo" + 0.011*"siempr" + 0.010*"nunca" + 0.009*"tiempo" + 0.008*"corazÃ³n" + 0.008*"ahora"'), (5, '0.044*"cest" + 0.033*"tout" + 0.028*"plus" + 0.026*"comm" + 0.020*"nous" + 0.017*"quand" + 0.014*"fait" + 0.014*"bien" + 0.013*"refrain" + 0.013*"rien"'), (6, '0.019*"light" + 0.013*"da

In [29]:
res2 = lda_model_3.get_document_topics(bow_corpus)

In [30]:
topic_count = []
for i in range(20):
    topic_count.append(0)
list1 = []
for i in range(len(res2)):
    a = []
    for c in res2[i]:
        if c[1] >0.24999:
            a.append(c[0])
    list1.append(a)
    for j in range(20):
        if j in a:
            topic_count[j] += 1
            
    if(i%10000 == 0):
        print(i)
print(topic_count)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
[3927, 17922, 4574, 46871, 15305, 6638, 31599, 3862, 5480, 13394, 5103, 8769, 506, 34511, 9236, 26340, 10071, 10432, 29265, 33937]


In [2]:
from gensim import corpora, models, similarities
final_lda_model =  models.LdaModel.load('lda_Model_topics_20_alpha_not_def')


In [4]:
print(final_lda_model.print_topics())

[(0, '0.075*"danc" + 0.040*"fire" + 0.028*"citi" + 0.025*"black" + 0.024*"burn" + 0.024*"move" + 0.022*"parti" + 0.019*"white" + 0.017*"water" + 0.015*"town"'), (1, '0.034*"like" + 0.018*"nigga" + 0.017*"fuck" + 0.017*"this" + 0.015*"shit" + 0.011*"caus" + 0.009*"money" + 0.009*"back" + 0.009*"bitch" + 0.009*"know"'), (2, '0.153*"gonna" + 0.119*"wanna" + 0.063*"give" + 0.063*"make" + 0.053*"gotta" + 0.025*"ride" + 0.024*"turn" + 0.023*"round" + 0.018*"take" + 0.017*"around"'), (3, '0.026*"would" + 0.016*"could" + 0.015*"home" + 0.013*"said" + 0.012*"like" + 0.012*"time" + 0.011*"this" + 0.011*"never" + 0.009*"night" + 0.009*"look"'), (4, '0.030*"amor" + 0.021*"quiero" + 0.019*"vida" + 0.015*"mÃ¡s" + 0.011*"solo" + 0.011*"siempr" + 0.010*"nunca" + 0.009*"tiempo" + 0.008*"corazÃ³n" + 0.008*"ahora"'), (5, '0.044*"cest" + 0.033*"tout" + 0.028*"plus" + 0.026*"comm" + 0.020*"nous" + 0.017*"quand" + 0.014*"fait" + 0.014*"bien" + 0.013*"refrain" + 0.013*"rien"'), (6, '0.019*"light" + 0.013*"da

In [14]:
res2 = final_lda_model.get_document_topics(bow_corpus)
res2[:10]

In [17]:
for i in range(50):
    print(res2[i])

[(6, 0.17815761), (13, 0.25032383), (16, 0.1745107), (17, 0.3792196)]
[(2, 0.03722329), (6, 0.07206821), (15, 0.32703635), (18, 0.27773196), (19, 0.27796048)]
[(0, 0.057546735), (11, 0.06538877), (13, 0.057743646), (14, 0.03649703), (18, 0.32731542), (19, 0.45068005)]
[(14, 0.46628705), (15, 0.32678574), (19, 0.19364421)]
[(0, 0.22953562), (3, 0.16064781), (6, 0.5937702)]
[(3, 0.28128988), (13, 0.3743993), (16, 0.04892652), (17, 0.103064045), (18, 0.17938478)]
[(0, 0.012503117), (1, 0.012503117), (2, 0.012503117), (3, 0.012503117), (4, 0.012503117), (5, 0.2781907), (6, 0.012503117), (7, 0.25079414), (8, 0.012503117), (9, 0.25846216), (10, 0.012503117), (11, 0.012503117), (12, 0.012503117), (13, 0.012503117), (14, 0.012503117), (15, 0.012503117), (16, 0.012503117), (17, 0.012503117), (18, 0.012503117), (19, 0.012503117)]
[(6, 0.43696934), (13, 0.10877824), (17, 0.0567816), (18, 0.23101218), (19, 0.1565884)]
[(0, 0.027438452), (1, 0.7060495), (11, 0.026348006), (14, 0.016953763), (15, 0.

In [18]:
""" Adding all the topics on our dataset """

' Adding all the topics on our dataset '

In [17]:
import pandas as pd
df = pd.read_csv("../Dataset/unlabelled_song_features.csv")
df.head()


,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,release_date,popularity
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,0.957,105.157,239627,3/1/1999,48
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,0.146,104.678,210867,1/1/2006,44
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,0.754,92.005,298573,1989,54
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,0.343,123.925,233947,2006,19
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,0.630,120.758,144947,1/23/1959,53


In [19]:
master_dict = {}
for i in range(len(res)):
    list_ = [0]*3
    for j in res[i]:
        list_[j[0]] = j[1]
    
    master_dict[msd_ids[i]] = list_


In [20]:
print(len(master_dict))

237662


In [21]:
f = open("../Dataset/Msd_id_to_spotify_id.pkl", "rb")
msdid_to_spid = pickle.load(f)
print(len(msdid_to_spid))
spid_to_msdid ={v: k for k, v in msdid_to_spid.items()}
print(len(spid_to_msdid))

213887
209108


In [22]:
s = set()
for i in msdid_to_spid.keys():
    if(msdid_to_spid[i] not in s):
        s.add(msdid_to_spid[i])
    else:
        print(msdid_to_spid[i])

6tGtBvK6DezcjbtUxXGyxe
6Lzu3SKH0LthYfjGC0INsY
6NuWsJSk8dq3kT0DZdFNKo
2nMghZvtLx6DDgTEHEsb4w
2gLFwnKgi5aPIKCo6tYzPR
0UzJq86HhxcMzisUcX2BcV
6pvfDHdFcjiIDXFxvDI879
5ueyLj6e6oVaTY0KQ6yLaA
3QJ5HpjabhvaXnBYcv1p9w
6rLqjzGV5VMLDWEnuUqi8q
49SL9f1Kz73p0bzsNPTY4l
2dLt7Zl6JAF3CqgRZAanLl
0Ls4WLphg1obbq7wTPiy7k
0sJOfQeaYEgA9UxMhDCtLm
4YskHNlSyJAanQ0kw2b5iz
0DTHza2fzvhVeiFmnmCNsh
5B23BmNt1R61I7b8iuB4jg
5gS0oMmjRMDn21Dduls4OC
2hl9ONQ7YGgZGYx0OSHvIq
6DHQaqFT9ldf8mOFulDBF6
7mAF2MJdbNT75VrVcgwT6F
77YyHUO5fX7YGxlNZm6uXh
2HeTmGTjl870ucJ8mF7zl5
7g7PQK3KFdjaCcwQGbfbAe
1c9dnQbOzw01ID7X2IsYOE
5ubqSAc9LxUS2CsMHcA4kF
5e6NiVezsdfeGuSIxFlTI8
5TridrikuB8m6Cc1Em7mKN
3xhPwNiagVUtTyqDv71MSU
4UoJKode5NkX9eo91hWSeD
4ThMItbDD7hZgCehNx2Qxy
4Jyx1EEo1aKX3IUJdw7DqK
0mTHshGGNTGxUiyohrdj2Z
3RxtFPq7H90WgyigfTZz08
0DTHza2fzvhVeiFmnmCNsh
0jNjI4KJleZP7EuzpzOvTo
676XDlJy1S5sOwrDKIqNvt
1jBfY6x1uIZTUxYAOi54c3
1jaZeL8i7KB4GzKsdEH9mH
6rB2zXq6Uf4YjbnlItSyMp
2FQ4GDCwqtA7pSGprpdhGs
2Cy7QY8HPLk925AyNAt6OG
5PAxUWPI4gyOuu4XhneRJU
1FMHNVeJ9s1

In [23]:
un_song_ids = list(df["sp_song_id"])
print(len(un_song_ids))

11161


In [24]:
list2 = []
for i in df["sp_song_id"]:
    if(i in spid_to_msdid ):
        if(spid_to_msdid[i] in master_dict):
            list2.append(master_dict[spid_to_msdid[i]])
print(len(list2))


11161


In [25]:
lda_topic_names = []
for i in range(3):
    lda_topic_names.append("lda_topic_" + str(i))
print(lda_topic_names)

['lda_topic_0', 'lda_topic_1', 'lda_topic_2']


In [26]:
df2 = pd.DataFrame(list2, columns =lda_topic_names )
df2.head()

,lda_topic_0,lda_topic_1,lda_topic_2
0,0.921150,0.078573,0.0
1,0.000000,0.998903,0.0
2,0.996161,0.000000,0.0
3,0.186477,0.813353,0.0
4,0.166753,0.832920,0.0


In [27]:
final_df = pd.concat([df,df2], axis = 1)


In [28]:
final_df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,release_date,popularity,lda_topic_0,lda_topic_1,lda_topic_2
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,0.957,105.157,239627,3/1/1999,48,0.921150,0.078573,0.0
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,0.146,104.678,210867,1/1/2006,44,0.000000,0.998903,0.0
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,0.754,92.005,298573,1989,54,0.996161,0.000000,0.0
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,0.343,123.925,233947,2006,19,0.186477,0.813353,0.0
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,0.630,120.758,144947,1/23/1959,53,0.166753,0.832920,0.0


In [29]:
final_df.to_csv("../Dataset/unlabelled_song_features_and_lda_topics_3.csv", index = False)